In [11]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("../data/eu_ai_act.pdf")
docs = loader.load()

print(f"문서의 페이지수: {len(docs)}")

문서의 페이지수: 144


In [12]:
print(docs[10].page_content)

input data directly acquired by an AI system in the course of the use of such system as well as the results and outputs 
of the use linked to that authorisation. It should not include input that is legally acquired in accordance with another 
Union or national law. In any case, no decision producing an adverse legal effect on a person should be taken based 
solely on the output of the remote biometric identification system.
(36)
In order to carry out their tasks in accordance with the requirements set out in this Regulation as well as in national 
rules, the relevant market surveillance authority and the national data protection authority should be notified of each 
use of the real-time biometric identification system. Market surveillance authorities and the national data protection 
authorities that have been notified should submit to the Commission an annual report on the use of real-time 
biometric identification systems.
(37)
Furthermore, it is appropriate to provide, within the ex

In [13]:
docs[10].__dict__

{'id': None,
 'metadata': {'producer': 'PDFlib+PDI 9.0.7p3 (C++/Win64)',
  'creator': 'Servigistics Arbortext Publishing Engine',
  'creationdate': '2024-07-11T14:47:17+02:00',
  'source': '../data/eu_ai_act.pdf',
  'file_path': '../data/eu_ai_act.pdf',
  'total_pages': 144,
  'format': 'PDF 1.7',
  'title': 'Regulation (EU) 2024/1689 of the European Parliament and of the Council of 13 June 2024 laying down harmonised rules on artificial intelligence and amending Regulations (EC) No 300/2008, (EU) No 167/2013, (EU) No 168/2013, (EU) 2018/858, (EU) 2018/1139 and (EU) 2019/2144 and Directives 2014/90/EU, (EU) 2016/797 and (EU) 2020/1828 (Artificial Intelligence Act)Text with EEA relevance.',
  'author': 'Publications Office of the European Union L-2985 Luxembourg LUXEMBOURG',
  'subject': 'I Legislative acts',
  'keywords': 'ISSN 1977-0677',
  'moddate': '2024-07-11T15:55:28+02:00',
  'trapped': '',
  'modDate': "D:20240711155528+02'00'",
  'creationDate': "D:20240711144717+02'00'",
  'p

In [14]:
docs[1].metadata

{'producer': 'PDFlib+PDI 9.0.7p3 (C++/Win64)',
 'creator': 'Servigistics Arbortext Publishing Engine',
 'creationdate': '2024-07-11T14:47:17+02:00',
 'source': '../data/eu_ai_act.pdf',
 'file_path': '../data/eu_ai_act.pdf',
 'total_pages': 144,
 'format': 'PDF 1.7',
 'title': 'Regulation (EU) 2024/1689 of the European Parliament and of the Council of 13 June 2024 laying down harmonised rules on artificial intelligence and amending Regulations (EC) No 300/2008, (EU) No 167/2013, (EU) No 168/2013, (EU) 2018/858, (EU) 2018/1139 and (EU) 2019/2144 and Directives 2014/90/EU, (EU) 2016/797 and (EU) 2020/1828 (Artificial Intelligence Act)Text with EEA relevance.',
 'author': 'Publications Office of the European Union L-2985 Luxembourg LUXEMBOURG',
 'subject': 'I Legislative acts',
 'keywords': 'ISSN 1977-0677',
 'moddate': '2024-07-11T15:55:28+02:00',
 'trapped': '',
 'modDate': "D:20240711155528+02'00'",
 'creationDate': "D:20240711144717+02'00'",
 'page': 1}

In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

print(f"분할된 청크의수: {len(split_documents)}")

분할된 청크의수: 1398


In [ ]:
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vector_db = FAISS.from_documents(split_documents, embeddings)
# 벡터 DB 저장하기
def save_vector_db(vector_db, path=".data/vectorstore"):
    """
    FAISS 벡터 데이터베이스를 지정된 경로에 저장합니다.
    
    Args:
        vector_db: FAISS 벡터 스토어 인스턴스
        path: 저장 경로 (기본값: "./vectorstore")
    """
    import os
    from loguru import logger
    
    try:
        # 디렉토리가 없으면 생성
        os.makedirs(path, exist_ok=True)
        
        # 벡터 스토어 저장
        vector_db.save_local(path)
        logger.info(f"벡터 데이터베이스가 성공적으로 저장되었습니다: {path}")
        return True
    except Exception as e:
        logger.error(f"벡터 데이터베이스 저장 중 오류 발생: {e}")
        return False
    
save_vector_db(vector_db, path = "./data/vectorstore")

2025-05-20 13:08:31.311 | INFO     | __main__:save_vector_db:24 - 벡터 데이터베이스가 성공적으로 저장되었습니다: ./data/vectorstore


True

In [32]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_db_path = "./data/vectorstore"

def load_vector_db(embeddings, vector_db_path):
    """벡터 데이터베이스를 로드합니다."""
    return FAISS(vector_db_path, embeddings)


In [37]:
# 임베딩 모델 설정
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 저장된 벡터 DB 경로
vector_db_path = "./data/vectorstore"

# 벡터 DB 로드
vector_db = FAISS.load_local(vector_db_path, embeddings, allow_dangerous_deserialization=True)

# Retriever 생성
retriever = vector_db.as_retriever()
query = '''High-risk AI systems recruitment 
AI agent requirements under EU AI Act Article 6, Article 9, Article 15, Article 52'''
retriever.invoke(query)

[Document(id='47f0dfbd-410a-4963-b0c2-1a9cad5ae34a', metadata={'producer': 'PDFlib+PDI 9.0.7p3 (C++/Win64)', 'creator': 'Servigistics Arbortext Publishing Engine', 'creationdate': '2024-07-11T14:47:17+02:00', 'source': '../data/eu_ai_act.pdf', 'file_path': '../data/eu_ai_act.pdf', 'total_pages': 144, 'format': 'PDF 1.7', 'title': 'Regulation (EU) 2024/1689 of the European Parliament and of the Council of 13 June 2024 laying down harmonised rules on artificial intelligence and amending Regulations (EC) No 300/2008, (EU) No 167/2013, (EU) No 168/2013, (EU) 2018/858, (EU) 2018/1139 and (EU) 2019/2144 and Directives 2014/90/EU, (EU) 2016/797 and (EU) 2020/1828 (Artificial Intelligence Act)Text with EEA relevance.', 'author': 'Publications Office of the European Union L-2985 Luxembourg LUXEMBOURG', 'subject': 'I Legislative acts', 'keywords': 'ISSN 1977-0677', 'moddate': '2024-07-11T15:55:28+02:00', 'trapped': '', 'modDate': "D:20240711155528+02'00'", 'creationDate': "D:20240711144717+02'00

In [16]:
retriever.invoke()

[]